# Complementaria Repaso Quiz 3 - Martes 12:30

In [14]:
import numpy as np
import pandas as pd
from jmarkov.mdp.dtmdp import dtmdp


In [15]:
# Matrices de transición (orden de estados: O, D, C)

P_S = np.array([
    [0.85, 0.15, 0.00],
    [0.60, 0.35, 0.05],
    [0.40, 0.40, 0.20]
])

P_M = np.array([
    [0.70, 0.25, 0.05],
    [0.25, 0.55, 0.20],
    [0.10, 0.40, 0.50]
])

P_N = np.array([
    [0.60, 0.30, 0.10],
    [0.10, 0.50, 0.40],
    [0.00, 0.20, 0.80]
])

# Ingresos por estado (USD por semana)
ingresos = {
    'O': 300,
    'D': 200,
    'C': 80
}

# Costos por acción (USD por semana)
costos = {
    'S': -120,   # Servicio preventivo
    'M': -60,    # Mantenimiento parcial
    'N': 0       # No intervenir
}


## Parte A: Implementación del MDP

In [16]:
# Factor de descuento
beta = 0.8

# Espacio de estados y acciones
# O: Óptima, D: Degradada, C: Crítica
estados = np.array(['O', 'D', 'C'], dtype=str)

# S: Servicio preventivo, M: Mantenimiento parcial, N: No intervenir
acciones = np.array(['S', 'M', 'N'], dtype=str)

# Diccionario de matrices de transición por acción
matrices = {}
for a in acciones:
    if a == 'S':
        matrices[a] = P_S
    elif a == 'M':
        matrices[a] = P_M
    else:  # 'N'
        matrices[a] = P_N

# Matriz de retornos R(s,a) = ingreso(s) + costo(a)
# filas: estados [O, D, C], columnas: acciones [S, M, N]
retornos = np.zeros((len(estados), len(acciones)), dtype=float)

for i, s in enumerate(estados):
    for j, a in enumerate(acciones):
        retornos[i, j] = ingresos[s] + costos[a]

retornos_df = pd.DataFrame(retornos, index=estados, columns=acciones)
retornos_df


,S,M,N
O,180.0,240.0,300.0
D,80.0,140.0,200.0
C,-40.0,20.0,80.0


In [17]:
mdp = dtmdp(estados, acciones, matrices, retornos, beta)
mdp

In [18]:
value_functions, optimal_policy = mdp.solve(0, minimize=False, method='value_iteration')

In [19]:
optimal_policy

{np.str_('O'): np.str_('N'),
 np.str_('D'): np.str_('S'),
 np.str_('C'): np.str_('S')}

In [20]:
value_functions

array([1089.16967505,  876.17328516,  701.08303245])

In [21]:
mdp.expected_policy_value(value_functions, optimal_policy)

np.float64(983.7209301945735)

In [22]:
matrizPO = mdp.policy_transition_matrix(optimal_policy)

# En DataFrame para indexar por letras de estado
matrizPO_df = pd.DataFrame(matrizPO, index=estados, columns=estados)
matrizPO_df

,O,D,C
O,0.6,0.30,0.10
D,0.6,0.35,0.05
C,0.4,0.40,0.20


## Parte B: Simulación de Monte Carlo

In [23]:
escenarios = 1500
semanas = 52

np.random.seed(0)

# Matriz para guardar la ganancia de cada semana y escenario
ganancia = np.zeros((semanas, escenarios), dtype=float)

# Matriz para marcar estados Críticos
indicador_C = np.zeros((semanas, escenarios), dtype=float)

for j in range(escenarios):
    
    # Estado inicial: Óptima (O)
    estado = 'O'
    
    for i in range(semanas):
        # Acción según política óptima
        accion = str(optimal_policy[estado])
        
        # Retorno inmediato en esa semana
        ganancia[i, j] = retornos_df.loc[estado, accion]
        
        # Marcar si la máquina está en estado Crítica (C)
        if estado == 'C':
            indicador_C[i, j] = 1
        
        # Transición al siguiente estado (bajo la política óptima)
        estado = np.random.choice(estados, p=matrizPO_df.loc[estado, :])

In [24]:
# Ganancia esperada en la última semana (semana 52)
ganancia_ultima_semana = ganancia[-1, :].mean()
ganancia_ultima_semana

np.float64(193.33333333333334)

In [25]:
# Porcentaje de semanas en las que la máquina está en estado Crítica (C)
porcentaje_C = indicador_C.mean() * 100
porcentaje_C


np.float64(9.007692307692308)